# Tutorial: **$\delta$ HBV 1.1p**

---

This notebook demonstrates how to train and forward the $\delta$ HBV 1.1p model developed by [Yalan Song et al. (2024)](https://essopenarchive.org/users/810569/articles/1213791-improving-physics-informed-differentiable-hydrologic-models-for-capturing-unseen-extreme-events?commit=9860e1a182de65bf97f387c3d7021868dcc52081). A pre-trained model is provided for those who only wish to run the model forward. For explanation of model structure, methodologies, data, and performance metrics, please refer to Song's publication [below](#publication). If you find this code is useful in your own work, please include the aforementioned citation.

<br>

### Before Running:
- **Environment**: From `env/` a minimal Python environment can be setup for running this code... (see `docs/getting_started.md` for more details.)
    - Conda -- `deltamodel_env.yaml`
    - Pip -- `requirements.txt`


- **Model**: The trained $\delta$ HBV 1.1p model can be downloaded from [AWS](https://mhpi-spatial.s3.us-east-2.amazonaws.com/mhpi-release/models/dHBV_1_1p_trained.zip). After downloading...

    - Update the `trained_model` key in model config `example/conf/config_dhbv_1_1p.yaml` with the path to you directory containing the trained model `dHBV_1_1p_Ep50.pt` (or *Ep100*) AND normalization `test1989-1999_Ep50/normalization_statistics.json`.

- **Data**: The CAMELS data extraction used in model training and evaluation can be downloaded from [AWS](https://mhpi-spatial.s3.us-east-2.amazonaws.com/mhpi-release/camels/camels_data.zip). After downloading, in the data configs `example/conf/observations/camels_531.yaml` and `camels_671.yaml` update...
    1. `train_path` key with your path to `training_file`,
    2. `test_path` with your path to `valication file`,
    3. `gage_info` with your path to `gage_ids.npy`,
    4. (CAMELS 531 only) `subset_path` with your path to `531_subset.txt`.

- **Hardware**: The LSTMs used in this model require CUDA support only available with Nvidia GPUs. For those without access, T4 GPUs can be used when running this notebook with dMG on [Google Colab](https://colab.research.google.com/).


    
### Publication:

*Yalan Song, Kamlesh Sawadekar, Jonathan M Frame, Ming Pan, Martyn Clark, Wouter J M Knoben, Andrew W Wood, Trupesh Patel, Chaopeng Shen. "Improving physics-informed, differentiable hydrologic models for capturing unseen extreme events." ESS Open Archive (2024). https://doi.org/10.22541/essoar.172304428.82707157/v1.*

<br>

### Issues:
For questions, concerns, bugs, etc., please reach out by posting an issue on the [dMG repo](https://github.com/mhpi/generic_deltaModel/issues).

---

<br>

## 1.1 Train $\delta$ HBV 1.1p

After completing [these](#before-running) steps (model file not needed), train a $\delta$ HBV 1.1p model with the code block below.

--> For default settings with 50 training epochs, expect train times of ~8 hours with an Nvidia RTX 3090.

**Note**
- The settings defined in the config `../example/conf/config_dhbv_1_1p.yaml` are set to replecate benchmark performance.
- For model training, set `mode: train` in the config, or modify after config dict has been created (see below).
- An `example/results/` directory will be generated to store experiment and model files. This location can be adjusted by changing the `save_path` key in your config. 
- If you are new to the *dMG* framework and want explanation of the methods used below, we suggest first looking at our notebook for $\delta$ HBV 1.0: `example/hydrology/example_dhbv_1_0.ipynb`.
- The default training window from 1 October 1999 to 30 September 2008 with `batch_size=100` should use ~2.8GB of vram.

In [ ]:
import sys
sys.path.append('../../')
sys.path.append('../../deltaModel')  # Add the dMG root directory.

from example import load_config 
from models.model_handler import ModelHandler as dHBV
from core.utils import print_config
from core.utils.module_loaders import get_data_loader, get_trainer



#------------------------------------------#
# Define model settings here.
CONFIG_PATH = '../example/conf/config_dhbv_1_1p.yaml'
#------------------------------------------#



# 1. Load configuration dictionary of model parameters and options.
config = load_config(CONFIG_PATH)
config['mode'] = 'train'  # <-- Confirm that we are doing training if not set in the config file.
print_config(config)

# 2. Initialize the differentiable HBV 1.1p model (LSTM + HBV 1.1p).
model = dHBV(config, verbose=True)

# 3. Load and initialize a dataset dictionary of NN and HBV model inputs.
data_loader = get_data_loader(config['data_loader'])
data_loader = data_loader(config, test_split=True, overwrite=False)

# 4. Initialize trainer to handle model training.
trainer = get_trainer(config['trainer'])
trainer = trainer(
    config,
    model,
    train_dataset = data_loader.train_dataset,
    verbose=True,
)

# 5. Start model training.
trainer.train()

## 1.2 Evaluate Model Performance

After completing the training in [1.1](#11-train--hbv-11p), or with the trained model provided, test $\delta$ HBV 1.1p below on the evaluation data.

--> For default settings expect evaluation time of ~5 minutes with an Nvidia RTX 3090.

**Note**
- For model evaluation, set `mode: test` in `example/conf/config_dhbv_1_1p.yaml`, or modify after the config dict has been created (see below).
- When evaluating provided models, confirm that `test: test_epoch` in the config corresponds to your desired model (50 or 100 epochs).
- The default evaluation window from 1 October 1989 to 30 September 1999 with `batch_size=25` should use ~2.7GB of vram.

In [ ]:
import sys
sys.path.append('../../')
sys.path.append('../../deltaModel')  # Add the dMG root directory.

from example import load_config 
from models.model_handler import ModelHandler as dHBV
from core.utils import print_config
from core.utils.module_loaders import get_data_loader, get_trainer



#------------------------------------------#
# Define model settings here.
CONFIG_PATH = '../example/conf/config_dhbv_1_1p.yaml'
#------------------------------------------#



# 1. Load configuration dictionary of model parameters and options.
config = load_config(CONFIG_PATH)
config['mode'] = 'test'  # <-- Confirm that we are doing testing if not set in the config file.
print_config(config)

# 2. Initialize the differentiable HBV 1.1p model (LSTM + HBV 1.1p).
model = dHBV(config, verbose=True)

# 3. Load and initialize a dataset dictionary of NN and HBV model inputs.
data_loader = get_data_loader(config['data_loader'])
data_loader = data_loader(config, test_split=True, overwrite=False)

# 4. Initialize trainer to handle model evaluation.
trainer = get_trainer(config['trainer'])
trainer = trainer(
    config,
    model,
    eval_dataset = data_loader.eval_dataset,
    verbose=True,
)

# 5. Start testing the model.
print('Evaluating model...')
trainer.evaluate()

### Visualizing Trained Model Performance

After running evaluation on the model, a new directory (e.g., for a model trained for 50 epochs and tested from years 1989-1999), `test1989-1999_Ep50/` will be created in the same directory containing the model files. This path will be populated with...

1. All model outputs (fluxes, states), including the target variable, *streamflow* (`flow_sim.npy`),

2. `flow_sim_obs`, streamflow observation data for comparison against model predictions,

2. `metrics.json`, containing evaluation metrics accross the test time range for every gage in the dataset,

3. `metrics_agg.json`, containing evaluation metrics statistics across all gages (mean, median, standar deviation).

4. `normalization_statistics.json`, containing statistics used for normalizing the testing data.


We can use these outputs to visualize $\delta$ HBV 1.1p's performance with a 
1. Cumulative distribution function (CDF) plot, 

2. CONUS map of gage locations and metric (e.g., NSE) performance.

<br>

But first, let's first check the (basin-)aggregated metrics for NSE, KGE, bias, RMSE, and, for both high/low flow regimes, RMSE and absolute percent bias...

In [ ]:
import os

from core.post import print_metrics
from core.data import load_json


print(f"Evaluation output files saved to: {config['out_path']} \n")


# 1. Load the basin-aggregated evaluation results.
metrics_path = os.path.join(config['out_path'], 'metrics_agg.json')
metrics = load_json(metrics_path)
print(f"Available metrics: {metrics.keys()} \n")

# 2. Print the evaluation results.
metric_names =  [
    # Choose metrics to show.
    'nse', 'kge', 'bias', 'rmse', 'rmse_low', 'rmse_high', 'flv_abs', 'fhv_abs',
]
print_metrics(metrics, metric_names, mode='median', precision=3)

#### Generate the CDF Plot

The cumulative distribution function (CDF) plot tells us what percentage (CDF on the y-axis) of basins performed at least better than a given metric on the evaluation data.
We give an example of such a plot below for NSE, but you can adjust this to your preferred metric. See the output from the previous cell to see what metrics are available. (Note some may require changing `xbounds` in the `plot_cdf`.)

In [ ]:
### Plot CDF of the evaluation results.
from core.post.plot_cdf import plot_cdf



#------------------------------------------#
# Choose the metric to plot. (See available metrics printed above, or in the metrics_agg.json file).
METRIC = 'nse'
#------------------------------------------#



# 1. Load the evaluation metrics.
metrics_path = os.path.join(config['out_path'], 'metrics.json')
metrics = load_json(metrics_path)

# 2. Plot the CDF for NSE.
plot_cdf(
    metrics=[metrics],
    metric_names=[METRIC],
    model_labels=['dHBV 1.1p'],
    title="CDF of NSE for $\delta$HBV 1.1p",
    xlabel=METRIC.capitalize(),
    figsize=(8, 6),
    xbounds=(0, 1),
    ybounds=(0, 1),
    show_arrow=True
)

#### Generate the Spatial Plot

This plot shows the locations of each basin in the evaluation data, color-coded by performance on a metric. Here we give a plot for NSE, but as before, this metric can be changed to your preference. (See above for what is available; for metrics not valued between 0 and 1, you will need to set `dynamic_colorbar=True` in `geoplot_single_metric` to ensure proper coding.)

In [ ]:
### Plot the evaluation results spatially.
import numpy as np
import pandas as pd
import geopandas as gpd

from core.data import txt_to_array
from core.post.plot_geo import geoplot_single_metric



#------------------------------------------#
# Choose the metric to plot. (See available metrics printed above, or in the metrics_agg.json file).
METRIC = 'nse'

# Set the paths to the gage id lists and shapefiles...
GAGE_ID_PATH = 'your/path/to/gageid.npy'
GAGE_ID_531_PATH = 'your/path/to/Sub531ID.txt'
SHAPEFILE_PATH = 'your/path/to/camels_671_loc.shp'
#------------------------------------------#



# 2. Load gage ids + basin shapefile with geocoordinates (lat, long) for every gage.
gage_ids = np.load(GAGE_ID_PATH, allow_pickle=True)
gage_ids_531 = txt_to_array(GAGE_ID_531_PATH)
coords = gpd.read_file(SHAPEFILE_PATH)

# 3. Format geocoords for 531- and 671-basin CAMELS sets.
coords_531 = coords[coords['gage_id'].isin(list(gage_ids_531))].copy()

coords['gage_id'] = pd.Categorical(coords['gage_id'], categories=list(gage_ids), ordered=True)
coords_531['gage_id'] = pd.Categorical(coords_531['gage_id'], categories=list(gage_ids_531), ordered=True)

coords = coords.sort_values('gage_id')  # Sort to match order of metrics.
basin_coords_531 = coords_531.sort_values('gage_id')

# 4. Load the evaluation metrics.
metrics_path = os.path.join(config['out_path'], 'metrics.json')
metrics = load_json(metrics_path)

# 5. Add the evaluation metrics to the basin shapefile.
if config['observations']['name'] == 'camels_671':
    coords[METRIC] = metrics[METRIC]
    full_data = coords
elif config['observations']['name'] == 'camels_531':
    coords_531[METRIC] = metrics[METRIC]
    full_data = coords_531
else:
    raise ValueError(f"Observation data supported: 'camels_671' or 'camels_531'. Got: {config['observations']}")

# 6. Plot the evaluation results spatially.
geoplot_single_metric(
    full_data,
    METRIC,
    f"Spatial Map of {METRIC.upper()} for $\delta$HBV 1.1p on CAMELS " \
        f"{config['observations']['name'].split('_')[-1]}",
    dynamic_colorbar=False,
)

## 2. Forward $\delta$ HBV 1.1p

After completing [these](#before-running) steps, forward the $\delta$ HBV 1.1p model with the code block below.

Note:
- The settings defined in `../example/conf/config_dhbv_1_1p.yaml` are set to replecate benchmark performance.
- The default inference window is set from 1 October 2012 to 30 September 2014, which should use ~2.7GB of vram.
- The first year (`warm_up` in the config, 365 days is default) of the inference period is used for initializing HBV's internal states (water storages) and is, therefore, excluded from the model's prediction output.

In [ ]:
import sys
sys.path.append('../../')
sys.path.append('../../deltaModel')  # Add the dMG root directory.

from example import load_config 
from models.model_handler import ModelHandler as dHBV
from core.utils import print_config
from core.utils.module_loaders import get_data_loader, get_trainer



#------------------------------------------#
# Define model settings here.
CONFIG_PATH = '../example/conf/config_dhbv_1_1p.yaml'
#------------------------------------------#



# 1. Load configuration dictionary of model parameters and options.
config = load_config(CONFIG_PATH)
config['mode'] = 'predict'  # <-- Confirm that we are doing training if not set in the config file.
print_config(config)

# 2. Initialize the differentiable HBV 1.1p model (LSTM + HBV 1.1p).
model = dHBV(config, verbose=True)

# 3. Load and initialize a dataset dictionary of NN and HBV model inputs.
data_loader = get_data_loader(config['data_loader'])
data_loader = data_loader(config, test_split=False, overwrite=False)
dataset = data_loader.dataset

# 4. Forward the model to get the predictions.
predictions = model.forward(dataset, eval=True)

### Visualizing Model Predictions

After running model inference we can, e.g., view the hydrograph for one of the basins to see we are getting expected outputs.

We can do this with our target variable, streamflow, for instance... (though, there are many other states and fluxes we can output as shown in the output cell below.)

In [ ]:
import numpy as np

from core.utils.dates import Dates
from core.data import txt_to_array
from core.post.plot_hydrograph import plot_hydrograph



#------------------------------------------#
# Choose a basin by USGS gage ID to plot.
GAGE_ID = 1022500
TARGET = 'flow_sim'

# Resample to 3-day prediction. Options: 'D', 'W', 'M', 'Y'.
RESAMPLE = '3D'

# Set the paths to the gage ID lists...
GAGE_ID_PATH = 'your/path/to/gageid.npy'
GAGE_ID_531_PATH = 'your/path/to/Sub531ID.txt'
#------------------------------------------#



print(f"HBV states and fluxes: {predictions['HBV_1_1p'].keys()} \n")


# 1. Get the streamflow predictions and daily timesteps of the prediction window.
pred = predictions['HBV_1_1p'][TARGET]
timesteps = Dates(config['predict'], config['dpl_model']['rho']).batch_daily_time_range

# Remove warm-up period to match model output (see Note above.)
timesteps = timesteps[config['dpl_model']['phy_model']['warm_up']:]


# 2. Load the gage ID lists and get the basin index.
gage_ids = np.load(GAGE_ID_PATH, allow_pickle=True)
gage_ids_531 = txt_to_array(GAGE_ID_531_PATH)

print(f"First 20 available gage IDs: \n {gage_ids[:20]} \n")
print(f"First 20 available gage IDs (531 subset): \n {gage_ids_531[:20]} \n")

if config['observations']['name'] == 'camels_671':
    if GAGE_ID in gage_ids:
        basin_idx = list(gage_ids).index(GAGE_ID)
    else:
        raise ValueError(f"Basin with gage ID {GAGE_ID} not found in the CAMELS 671 dataset.")

elif config['observations']['name'] == 'camels_531':
    if GAGE_ID in gage_ids_531:
        basin_idx = list(gage_ids_531).index(GAGE_ID)
    else:
        raise ValueError(f"Basin with gage ID {GAGE_ID} not found in the CAMELS 531 dataset.")
else:
    raise ValueError(f"Observation data supported: 'camels_671' or 'camels_531'. Got: {config['observations']}")


# 3. Get the data for the chosen basin and plot.
streamflow_pred_basin = pred[:, basin_idx].squeeze()

plot_hydrograph(
    timesteps,
    streamflow_pred_basin,
    streamflow_pred_basin,
    resample=RESAMPLE,
    title=f"Hydrograph for Gage ID {GAGE_ID}",
    ylabel='Streamflow (ft$^3$/s)',
)